In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')

In [39]:
prompt_text = "Given this product title, please select between 3 and 6 criteria to rate in order to compose a product review. No need to explain the criteria."
product_text = 'Maybelline Instant Age Rewind Eraser Dark Circles Treatment Concealer'

### Chunking

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def get_text_chunks(text) -> list[Document]:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
    return docs

In [61]:
texts = get_text_chunks('iphone 15')
texts

[Document(page_content='iphone 15')]

### QA Retrieval chain

In [27]:
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains.retrieval_qa.base import BaseRetrievalQA
from langchain.chains import RetrievalQA


OPENAI_MODEL = 'gpt-3.5-turbo'
BASE_PERSIST_DIRECTORY = '../db/chroma_3/'


def embed_texts(texts, model_name) -> list[Document]:
    if model_name == 'openai':
        embedder = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
    else:
        embedder = HuggingFaceEmbeddings(model_name=model_name)
        
    vector_db = Chroma.from_documents(
        texts, 
        embedder,
        persist_directory=BASE_PERSIST_DIRECTORY + model_name
    )
    return vector_db

def get_qa(model, vector_db) -> BaseRetrievalQA:
    qa = RetrievalQA.from_chain_type(
        llm=model,
        chain_type="stuff",
        retriever=vector_db.as_retriever(search_kwargs={"k": 1}),
    return_source_documents=False,
    verbose=False,
    )
    return qa

#### OpenAI

In [41]:
from langchain_openai import ChatOpenAI

llm_openai = ChatOpenAI(model_name=OPENAI_MODEL, openai_api_key=OPENAI_API_KEY)
print(llm_openai, '\n')

vector_db = embed_texts(texts, model_name='openai')
qa_openai = get_qa(llm_openai, vector_db)
print(qa_openai, '\n')

answer = qa_openai.invoke(prompt_text)
print(answer["result"])

client=<openai.resources.chat.completions.Completions object at 0x10a235db0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12ee8d1e0> openai_api_key=SecretStr('**********') openai_proxy=''
combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x10a235db0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12ee8d1e0>, openai_api_key=SecretStr('**********'), openai_proxy='')), document_v

#### Local

In [31]:
HG_EMBEDDING_MODEL = 'sentence-transformers/all-MiniLM-L6-v2'
MODEL_NAME = 'mistral-7b-openorca.gguf2.Q4_0.gguf'  # GPT4All model
MODEL_PATH = '../models/' + MODEL_NAME
!curl -C - -o {MODEL_PATH} https://gpt4all.io/models/gguf/{MODEL_NAME}

** Resuming transfer from byte position 4108928128
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27242    0 27242    0     0    97k      0 --:--:-- --:--:-- --:--:--   97k


In [55]:
from langchain.callbacks.streaming_stdout_final_only import FinalStreamingStdOutCallbackHandler
from langchain.llms import GPT4All

callbacks = [FinalStreamingStdOutCallbackHandler()]
llm_local = %time GPT4All(model=MODEL_PATH, callbacks=callbacks, verbose=True)
print(llm_local, '\n')

vector_db = %time embed_texts(texts, model_name=HG_EMBEDDING_MODEL)
qa_local = get_qa(llm_local, vector_db)
print(qa_local, '\n')

answer = %time qa_local.invoke(prompt_text)
print(answer["result"])

CPU times: user 149 ms, sys: 675 ms, total: 824 ms
Wall time: 865 ms
GPT4All
Params: {'model': '../models/mistral-7b-openorca.gguf2.Q4_0.gguf', 'max_tokens': 200, 'n_predict': 256, 'top_k': 40, 'top_p': 0.1, 'temp': 0.7, 'n_batch': 8, 'repeat_penalty': 1.18, 'repeat_last_n': 64, 'streaming': False} 

CPU times: user 900 ms, sys: 244 ms, total: 1.14 s
Wall time: 2.27 s
combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\n{context}\n\nQuestion: {question}\nHelpful Answer:"), llm=GPT4All(verbose=True, callbacks=[<langchain.callbacks.streaming_stdout_final_only.FinalStreamingStdOutCallbackHandler object at 0x10adaae60>], model='../models/mistral-7b-openorca.gguf2.Q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x12f92d8a0>)), document_vari

### LLM Chain

In [56]:
from langchain.prompts import PromptTemplate

prompt_text2 = "Please list between 3 and 6 criteria that could be rated for this product. Just return the list of criteria, nothing more."
template = f"Product: '{{product_text}}'\n{prompt_text2}"
prompt = PromptTemplate(template=template, input_variables=["product_text"])

In [57]:
from langchain.chains import LLMChain


llm_chain = %time LLMChain(prompt=prompt, llm=llm_local, return_final_only=True)
print(llm_chain, '\n')

res = %time llm_chain.run(product_text=PRODUCT_INPUT)
print(res)

CPU times: user 176 µs, sys: 2 µs, total: 178 µs
Wall time: 183 µs
prompt=PromptTemplate(input_variables=['product_text'], template="Product: '{product_text}'\nPlease list between 3 and 6 criteria that could be rated for this product. Just return the list of criteria, nothing more.") llm=GPT4All(verbose=True, callbacks=[<langchain.callbacks.streaming_stdout_final_only.FinalStreamingStdOutCallbackHandler object at 0x10adaae60>], model='../models/mistral-7b-openorca.gguf2.Q4_0.gguf', client=<gpt4all.gpt4all.GPT4All object at 0x12f92d8a0>) 

CPU times: user 4min 45s, sys: 2.85 s, total: 4min 48s
Wall time: 1min 13s

1. Coverage
2. Ease of application
3. Blendability
4. Longevity
5. Skin compatibility
6. Price

Please provide a brief description of each criterion:

1. Coverage - The amount of product needed to effectively cover dark circles and imperfections on the skin.
2. Ease of application - How easily the concealer can be applied, including its consistency and packaging design.
3. Ble

In [59]:
from langchain_openai import OpenAI


llm = OpenAI(openai_api_key=OPENAI_API_KEY)
llm_chain = LLMChain(prompt=prompt, llm=llm, return_final_only=True)
llm_chain.run(product_text=PRODUCT_INPUT)

'\n\n1. Coverage\n2. Longevity\n3. Shade range\n4. Consistency\n5. Ease of application\n6. Anti-aging benefits'